## Set up TG clients

In [1]:
import os

from dotenv import load_dotenv
from telethon import TelegramClient, functions
from telethon.sessions import StringSession


In [2]:
scrapers_creds = []

for i in range(1, 8):
    load_dotenv(f"../creds/scraper{i}.env", override=True)
    scrapers_creds.append(
        {
            "session": StringSession(os.getenv("TG_SESSION")),
            "api_id": int(os.getenv("TG_API_ID")),
            "api_hash": os.getenv("TG_API_HASH"),
            "device_model": os.getenv("TG_DEVICE_MODEL"),
            "system_version": os.getenv("TG_SYSTEM_VERSION"),
            "app_version": os.getenv("TG_APP_VERSION"),
        }
    )

tg_clients = {}

for creds in scrapers_creds:
    tg_clients[creds["api_id"]] = TelegramClient(**creds)
    await tg_clients[creds["api_id"]].connect()

## Get channels

In [3]:
import sys

sys.path.insert(0, "../scraper/")

In [4]:
load_dotenv("../creds/db.env", override=True)

True

In [5]:
from database.database import (
    Channels,
    Peers,
)
from database.session import get_database_session
from sqlalchemy import select
from telethon.errors import FloodWaitError
from telethon.types import InputPeerChannel
from tqdm import tqdm

In [6]:
# Get channels_id list
with get_database_session() as db_session:
    result = db_session.execute(select(Channels.id, Channels.name))
    chs = result.all()

len(chs)

26782

## Scrape pinned messages

In [7]:
ch_id_to_pinned_text = {}

In [8]:
peer_missed_channels = []

for ch_id, ch_name in tqdm(chs):
    channel = None  # Channel peer will be stored here
    tg_client = None  # Telegram client will be stored here, picked by scraper_id

    try:
        # Get channel peer, corresponding to the channel id and pick matching scraper (tg_client)
        with get_database_session() as db_session:
            result = db_session.execute(select(Peers).where(Peers.channel_id == ch_id))
            peer = result.scalars().all()

            # If don't have any peers for this channel, call ResolveUsername
            if not peer:
                print(f"Peer don't found for {ch_name}")
                scraper_id = None
                # Iterate over all scrapers, while don't get peer
                for scraper_id, tg_client in tg_clients.items():
                    try:
                        channel = await tg_client.get_input_entity(ch_name)
                        break
                    except FloodWaitError:
                        print(f"FWE for channel: {ch_name}, scraper: {scraper_id}")

                if not channel:
                    peer_missed_channels.append((ch_id, ch_name))
                    print("PEER MISSED CHANNEL")
                    continue

                # Save new peer data to the database
                new_peer = Peers(
                    scraper_id=scraper_id,
                    channel_name=ch_name,
                    channel_id=ch_id,
                    access_hash=channel.access_hash,
                )
                db_session.add(new_peer)

            else:
                channel = InputPeerChannel(
                    channel_id=peer[0].channel_id, access_hash=peer[0].access_hash
                )
                tg_client = tg_clients[peer[0].scraper_id]

        full = await tg_client(functions.channels.GetFullChannelRequest(channel))

        pinned_id = full.full_chat.pinned_msg_id
        if not pinned_id:
            continue

        async for post in tg_client.iter_messages(entity=channel, limit=1, ids=pinned_id):
            if post:
                # Save to dict
                ch_id_to_pinned_text[ch_id] = post.raw_text

    except Exception as e:
        print(f"Error while trying to scrape pinned message from {ch_id}")
        print(e)
        continue

print(f"Get posts: {len(ch_id_to_pinned_text)}, missed: {len(peer_missed_channels)}")

  0%|          | 0/26782 [00:00<?, ?it/s]

  0%|          | 2/26782 [00:00<2:51:30,  2.60it/s]

Error while trying to scrape pinned message from 2293523024
The channel specified is private and you lack permission to access it. Another reason may be that you were banned from it (caused by GetFullChannelRequest)
Error while trying to scrape pinned message from 2065156241
The channel specified is private and you lack permission to access it. Another reason may be that you were banned from it (caused by GetFullChannelRequest)


  0%|          | 3/26782 [00:01<2:02:47,  3.63it/s]

Error while trying to scrape pinned message from 1127716443
The channel specified is private and you lack permission to access it. Another reason may be that you were banned from it (caused by GetFullChannelRequest)


...

Error while trying to scrape pinned message from 1663211819
The channel specified is private and you lack permission to access it. Another reason may be that you were banned from it (caused by GetFullChannelRequest)


 95%|█████████▌| 25455/26782 [2:33:55<04:48,  4.61it/s]

Error while trying to scrape pinned message from 2046218676
The channel specified is private and you lack permission to access it. Another reason may be that you were banned from it (caused by GetFullChannelRequest)


100%|██████████| 26782/26782 [2:39:36<00:00,  2.80it/s]  

Get posts: 17938, missed: 0


In [9]:
import pickle

with open("/home/deniskirbaba/Documents/influai-data/embeddings/pinned_raw_texts.pkl", "wb") as f:
    pickle.dump(ch_id_to_pinned_text, f)